In [ ]:
import pandas as pd
file_path = "/home/sagemaker-user/TrajCL/data/porto_20200"
trajs = pd.read_pickle(file_path)

In [ ]:
trajs.head()

In [ ]:
# from torch.utils.data import Dataset
# class TrajDataset(Dataset):
#     def __init__(self, data):
#         # data: DataFrame
#         self.data = data

#     def __getitem__(self, index):
#         return self.data.loc[index].merc_seq

#     def __len__(self):
#         return self.data.shape[0]

In [ ]:
# l = trajs.shape[0]
# train_idx = (int(l*0), 200000)
# eval_idx = (int(l*0.7), int(l*0.8))
# test_idx = (int(l*0.8), int(l*1.0))

# train = TrajDataset(trajs[train_idx[0]: train_idx[1]])
# eval = TrajDataset(trajs[eval_idx[0]: eval_idx[1]])
# test = TrajDataset(trajs[test_idx[0]: test_idx[1]])

In [26]:
idx = 15
lon_lat_list = trajs['wgs_seq'][idx]

In [27]:
lon_lat_list

[[-8.660646, 41.168574],
 [-8.661087, 41.167926],
 [-8.661231, 41.166576],
 [-8.660637, 41.166396],
 [-8.660295, 41.166819],
 [-8.658954, 41.168394],
 [-8.657649, 41.169906],
 [-8.656371, 41.171454],
 [-8.654706, 41.173479],
 [-8.653014, 41.17527],
 [-8.651349, 41.17644],
 [-8.652213, 41.177241],
 [-8.651358, 41.178069],
 [-8.65071, 41.178924],
 [-8.65125, 41.17968],
 [-8.649648, 41.180643],
 [-8.647515, 41.18193],
 [-8.644491, 41.183541],
 [-8.641701, 41.184108],
 [-8.638677, 41.182452],
 [-8.635284, 41.180589],
 [-8.632476, 41.179104],
 [-8.629983, 41.179365],
 [-8.629857, 41.179374],
 [-8.629857, 41.179374],
 [-8.629263, 41.179437],
 [-8.625996, 41.179779],
 [-8.622018, 41.180463],
 [-8.61885, 41.181912],
 [-8.617077, 41.182821],
 [-8.614215, 41.183082],
 [-8.613675, 41.183136],
 [-8.613702, 41.183145],
 [-8.613288, 41.18319],
 [-8.610813, 41.184],
 [-8.607474, 41.184648],
 [-8.604018, 41.183892],
 [-8.601768, 41.183253],
 [-8.601588, 41.182632],
 [-8.601957, 41.181831],
 [-8.601903

In [ ]:
# !pip install folium
import folium

from folium import plugins
coordinates = [[x[0],x[1]] for x in lon_lat_list]
# Initialize map
m = folium.Map(location=coordinates[0], zoom_start=30)

folium.Marker(
    location=coordinates[0],
    popup="Start Location",
    icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
).add_to(m)
# Add markers
for lat, lon in coordinates[1:]:
    folium.Marker([lat, lon]).add_to(m)

# Draw arrows between points
for i in range(len(coordinates) - 1):
    start = coordinates[i]
    end = coordinates[i + 1]

    # Draw the line
    line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

    # Add directional arrow
    plugins.PolyLineTextPath(
        line,
        '➤',  # arrow symbol
        repeat=True,
        offset=7,
        attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
    ).add_to(m)

# Save and show the map
m.save("map_with_arrows.html")
m


In [1]:
import sys
sys.path.append("..")
from utils.cellspace import CellSpace

In [4]:
import pickle

porto_cs = pickle.load(open("/home/sagemaker-user/TrajCL/data/porto_20200_cell100_cellspace.pkl","rb"))


In [5]:
porto_cs

In [7]:
porto_cs.size()

36251

In [8]:
lon_lat_list = [[-8.660646, 41.168574],
 [-8.661087, 41.167926],
 [-8.661231, 41.166576],
 [-8.660637, 41.166396],
 [-8.660295, 41.166819],
 [-8.658954, 41.168394],
 [-8.657649, 41.169906],
 [-8.656371, 41.171454],
 [-8.654706, 41.173479],
 [-8.653014, 41.17527],
 [-8.651349, 41.17644],
 [-8.652213, 41.177241],
 [-8.651358, 41.178069],
 [-8.65071, 41.178924],
 [-8.65125, 41.17968],
 [-8.649648, 41.180643],
 [-8.647515, 41.18193],
 [-8.644491, 41.183541],
 [-8.641701, 41.184108],
 [-8.638677, 41.182452],
 [-8.635284, 41.180589],
 [-8.632476, 41.179104],
 [-8.629983, 41.179365],
 [-8.629857, 41.179374],
 [-8.629857, 41.179374],
 [-8.629263, 41.179437],
 [-8.625996, 41.179779],
 [-8.622018, 41.180463],
 [-8.61885, 41.181912],
 [-8.617077, 41.182821],
 [-8.614215, 41.183082],
 [-8.613675, 41.183136],
 [-8.613702, 41.183145],
 [-8.613288, 41.18319],
 [-8.610813, 41.184],
 [-8.607474, 41.184648],
 [-8.604018, 41.183892],
 [-8.601768, 41.183253],
 [-8.601588, 41.182632],
 [-8.601957, 41.181831],
 [-8.601903, 41.181795],
 [-8.601948, 41.181822],
 [-8.601894, 41.181813]]

In [10]:
porto_cs.x_min, porto_cs.x_max

(-969035.2296468767, -947853.0059660561)

In [11]:
from utils.tool_funcs import lonlat2meters

In [17]:
lonlat2meters(lon_lat_list[0][0],lon_lat_list[0][1])

(-964098.7026608016, 5037238.177438596)

In [14]:
point_ids = [porto_cs.get_cellid_by_point(*lonlat2meters(x[0],x[1])) for x in lon_lat_list]

In [15]:
point_ids

[8485,
 8313,
 8311,
 8481,
 8482,
 8826,
 9000,
 9344,
 9518,
 9863,
 10206,
 10037,
 10209,
 10381,
 10211,
 10555,
 10898,
 11585,
 12099,
 12609,
 13290,
 13801,
 14315,
 14315,
 14315,
 14486,
 14999,
 15855,
 16370,
 16714,
 17398,
 17398,
 17398,
 17569,
 17913,
 18597,
 19280,
 19621,
 19791,
 19619,
 19619,
 19619,
 19619]

In [1]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# List available shares.
shares = ddl.list_shares()

# Print shares.
print(shares)

[Table(name='bank_pending_transaction_overall_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='bank_connection_institutionid_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='login_ios_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='cs_audit_type_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='earnings_if_model_features_v5', share='sagemaker_share_prod', schema='ml_features'), Table(name='bank_active_bfpba_casino_spending_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='payroll_hourly_rate_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='bank_service_entertainment_transaction_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='bank_pending_service_insurance_transaction_features_pst6am', share='sagemaker_share_prod', schema='ml_features'), Table(name='ban

In [13]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_train_filtered")

# Display the loaded DataFrame.
df.count()

25/08/05 06:29:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


9069952

In [8]:
count=0
for row in df.toLocalIterator():
    print(row)
    count+=1
    if count>10:
        break

Row(traj_id='4474029_2024-01-05', timestamps_filtered=[datetime.datetime(2024, 1, 5, 8, 48, 58), datetime.datetime(2024, 1, 5, 9, 4, 57), datetime.datetime(2024, 1, 5, 9, 17, 28), datetime.datetime(2024, 1, 5, 9, 27, 36), datetime.datetime(2024, 1, 5, 9, 43, 13), datetime.datetime(2024, 1, 5, 10, 25, 28), datetime.datetime(2024, 1, 5, 10, 39, 36), datetime.datetime(2024, 1, 5, 11, 4, 40), datetime.datetime(2024, 1, 5, 11, 28, 30), datetime.datetime(2024, 1, 5, 11, 43), datetime.datetime(2024, 1, 5, 12, 14, 31), datetime.datetime(2024, 1, 5, 12, 34, 3), datetime.datetime(2024, 1, 5, 13, 49, 30), datetime.datetime(2024, 1, 5, 14, 39, 3), datetime.datetime(2024, 1, 5, 15, 55, 29), datetime.datetime(2024, 1, 5, 18, 6, 15), datetime.datetime(2024, 1, 5, 18, 20, 59), datetime.datetime(2024, 1, 5, 18, 42, 22), datetime.datetime(2024, 1, 5, 18, 57, 45), datetime.datetime(2024, 1, 5, 19, 24, 54), datetime.datetime(2024, 1, 5, 20, 42, 50), datetime.datetime(2024, 1, 5, 21, 4, 54), datetime.datet

In [10]:
row['traj_id']

'13883676_2024-01-07'

In [11]:
splits = df.randomSplit([0.8, 0.1, 0.1], seed=42)  # seed for reproducibility
train_df, val_df, test_df = splits

In [12]:
train_df.show()

+-------------------+--------------------+--------------------+--------------------+-------+
|            traj_id| timestamps_filtered|    wgs_seq_filtered|   merc_seq_filtered|trajlen|
+-------------------+--------------------+--------------------+--------------------+-------+
| 1000082_2022-04-10|[2022-04-10 07:29...|[[-74.08274056396...|[[-8246852.956151...|     31|
|10002344_2023-10-20|[2023-10-20 00:08...|[[-74.23185547444...|[[-8263452.352055...|     21|
|10002706_2024-04-29|[2024-04-29 06:06...|[[-73.71778276205...|[[-8206226.039481...|     32|
|10003339_2020-03-09|[2020-03-09 00:23...|[[-73.7786469, 40...|[[-8213001.404324...|     35|
|10003339_2021-11-23|[2021-11-23 00:17...|[[-73.7786388, 40...|[[-8213000.502636...|     27|
|10003826_2021-09-10|[2021-09-10 08:16...|[[-73.77224340299...|[[-8212288.570298...|     25|
|10004077_2022-07-18|[2022-07-18 07:56...|[[-74.21950644028...|[[-8262077.663860...|     27|
| 1000447_2019-07-18|[2019-07-18 02:14...|[[-74.15734039538...|[[-8255

In [13]:
df_with_index = df.rdd.zipWithIndex().map(lambda x: (x[1], x[0])).toDF(["row_index", "row"])

In [22]:
df_with_index.filter(df_with_index['row_index']==2).collect()[0]

Row(row_index=2, row=Row(traj_id='14252868_2024-01-05', timestamps_filtered=[datetime.datetime(2024, 1, 5, 0, 0, 48), datetime.datetime(2024, 1, 5, 8, 57, 29), datetime.datetime(2024, 1, 5, 9, 33, 4), datetime.datetime(2024, 1, 5, 9, 57, 17), datetime.datetime(2024, 1, 5, 10, 10, 58), datetime.datetime(2024, 1, 5, 10, 21, 48), datetime.datetime(2024, 1, 5, 10, 43, 52), datetime.datetime(2024, 1, 5, 11, 50, 57), datetime.datetime(2024, 1, 5, 12, 2, 31), datetime.datetime(2024, 1, 5, 12, 13, 16), datetime.datetime(2024, 1, 5, 12, 24), datetime.datetime(2024, 1, 5, 12, 36, 16), datetime.datetime(2024, 1, 5, 12, 47, 38), datetime.datetime(2024, 1, 5, 13, 1, 56), datetime.datetime(2024, 1, 5, 13, 28, 46), datetime.datetime(2024, 1, 5, 14, 35, 8), datetime.datetime(2024, 1, 5, 17, 16, 34), datetime.datetime(2024, 1, 5, 17, 57, 38), datetime.datetime(2024, 1, 5, 18, 26, 6), datetime.datetime(2024, 1, 5, 18, 42, 19), datetime.datetime(2024, 1, 5, 18, 56, 39), datetime.datetime(2024, 1, 5, 20, 

In [26]:
next(df.toLocalIterator())

Row(traj_id='4474029_2024-01-05', timestamps_filtered=[datetime.datetime(2024, 1, 5, 8, 48, 58), datetime.datetime(2024, 1, 5, 9, 4, 57), datetime.datetime(2024, 1, 5, 9, 17, 28), datetime.datetime(2024, 1, 5, 9, 27, 36), datetime.datetime(2024, 1, 5, 9, 43, 13), datetime.datetime(2024, 1, 5, 10, 25, 28), datetime.datetime(2024, 1, 5, 10, 39, 36), datetime.datetime(2024, 1, 5, 11, 4, 40), datetime.datetime(2024, 1, 5, 11, 28, 30), datetime.datetime(2024, 1, 5, 11, 43), datetime.datetime(2024, 1, 5, 12, 14, 31), datetime.datetime(2024, 1, 5, 12, 34, 3), datetime.datetime(2024, 1, 5, 13, 49, 30), datetime.datetime(2024, 1, 5, 14, 39, 3), datetime.datetime(2024, 1, 5, 15, 55, 29), datetime.datetime(2024, 1, 5, 18, 6, 15), datetime.datetime(2024, 1, 5, 18, 20, 59), datetime.datetime(2024, 1, 5, 18, 42, 22), datetime.datetime(2024, 1, 5, 18, 57, 45), datetime.datetime(2024, 1, 5, 19, 24, 54), datetime.datetime(2024, 1, 5, 20, 42, 50), datetime.datetime(2024, 1, 5, 21, 4, 54), datetime.datet

In [17]:
df.repartition(100).write.parquet("/home/sagemaker-user/TrajCL/data/parquet_files/nyc_v2/", mode="overwrite")

In [18]:
test = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/nyc_v2/part-00000-76372e3b-95b4-498e-9e85-2765482fcef5-c000.snappy.parquet")
test.head()

,traj_id,timestamps_filtered,wgs_seq_filtered,merc_seq_filtered,trajlen
0,3622855_2021-02-15,"[2021-02-15T09:37:10.000000000, 2021-02-15T09:...","[[-74.1403259, 40.8855913], [-74.1402448, 40.8...","[[-8253263.326435352, 4995481.024627576], [-82...",24
1,11026333_2025-07-03,"[2025-07-03T00:03:10.000000000, 2025-07-03T01:...","[[-73.9273032, 40.647141], [-73.9272849, 40.64...","[[-8229549.747943943, 4960433.5368635375], [-8...",28
2,2842506_2019-07-04,"[2019-07-04T07:35:24.000000000, 2019-07-04T07:...","[[-73.92489536229724, 40.677154541015625], [-7...","[[-8229281.708676959, 4964838.027314429], [-82...",28
3,2622694_2023-11-02,"[2023-11-02T05:05:01.000000000, 2023-11-02T05:...","[[-73.9493915, 40.8020117], [-73.9380027, 40.7...","[[-8232008.606252433, 4983182.150488543], [-82...",20
4,12427260_2022-08-25,"[2022-08-25T00:44:14.000000000, 2022-08-25T00:...","[[-74.0729984, 40.8964652], [-74.0741302, 40.8...","[[-8245768.463418968, 4997082.277930495], [-82...",47


In [19]:
len(test)

90699

In [61]:
import numpy as np
np.stack(test['merc_seq_filtered'].values[0])

array([[-8235771.75906886,  4976697.88178978],
       [-8236370.43592437,  4975334.56652866],
       [-8236181.04907908,  4968534.28471047],
       [-8234087.40250184,  4965895.23835057],
       [-8225433.6762035 ,  4964179.56799737],
       [-8221535.56588791,  4965200.87077021],
       [-8218174.3767914 ,  4959943.19536906],
       [-8213725.53165887,  4952348.28723248],
       [-8210199.33429042,  4954191.64843742],
       [-8210247.43345669,  4953524.95594323],
       [-8212152.09407179,  4952934.36691824],
       [-8212675.50408182,  4952509.34380704],
       [-8213456.80793909,  4952491.44850141],
       [-8217674.69883329,  4957342.27223223],
       [-8222657.34668965,  4964788.75331484],
       [-8230865.44145746,  4965241.18754029],
       [-8236553.30143016,  4968090.66007382],
       [-8239035.34663933,  4969175.11137875],
       [-8239841.3753153 ,  4959746.19938839],
       [-8246307.06831613,  4954433.27973355],
       [-8251783.37311906,  4954849.43648825],
       [-8251

In [6]:
import os
import torch
from torch.utils.data import IterableDataset
import pandas as pd
import numpy as np
class ParquetDataset(IterableDataset):
    def __init__(self, data_dir):
        self.files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".parquet")]

    def __iter__(self):
        for file_path in self.files:
            df = pd.read_parquet(file_path)
            features = df["merc_seq_filtered"].values
            for feature in features:
                yield torch.tensor(np.stack(feature))

In [7]:
parquet_dataset = ParquetDataset("/home/sagemaker-user/TrajCL/data/parquet_files/nyc/")

In [8]:
from torch.utils.data import DataLoader
dl = DataLoader(parquet_dataset, batch_size=1)

In [10]:
count = 0
for batch in dl:
    print(batch)
    count+=1
    if count>10:
        break

tensor([[[-8250434.3753,  4991187.6735],
         [-8250453.4666,  4991182.0660],
         [-8250453.8897,  4991195.4888],
         [-8250451.1289,  4991193.4430],
         [-8250450.0714,  4991193.2664],
         [-8250451.3070,  4991193.5313],
         [-8250455.7932,  4991177.6212],
         [-8252670.9398,  4993177.5981],
         [-8251117.8770,  4985989.8353],
         [-8252532.0687,  4991607.8216],
         [-8252540.2729,  4991602.4787],
         [-8250419.4697,  4992039.9692],
         [-8250438.8838,  4991198.7267],
         [-8250460.1235,  4991265.0462],
         [-8250467.3704,  4991177.4740],
         [-8251481.5578,  4994630.6660],
         [-8254385.9056,  4995512.8442],
         [-8252394.8897,  4996846.2238],
         [-8250451.1401,  4991194.2378],
         [-8250435.9338,  4991198.4177],
         [-8248785.6224,  4992633.8060],
         [-8251893.8407,  4992282.1922],
         [-8250450.4276,  4991191.5002],
         [-8250449.9489,  4991166.1559],
         [-82504

In [ ]:
tensor([[[-8250434.3753,  4991187.6735],